## 1.

Можно использовать классификатор, который пытается определить, из какого набора данных взят образец. Если модель не способна отличить тренинг от теста (точность близка к 50%), разбиение считается репрезентативным

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

# 1 - тест, 0 - трейн
X_combined = np.vstack([X_train, X_test])
labels = np.hstack([np.zeros(len(X_train)), np.ones(len(X_test))])

X_train_meta, X_test_meta, y_train_meta, y_test_meta = train_test_split(
    X_combined, labels, test_size=0.3, stratify=labels
)

clf = RandomForestClassifier()
clf.fit(X_train_meta, y_train_meta)
pred = clf.predict(X_test_meta)
score = accuracy_score(y_test_meta, pred)

print("Accuracy:", score)


Accuracy: 0.5555555555555556


## 2.

Для клиента не из кластера 2 найти такие изменения его признаков, чтобы он приблизился к центроиду кластера 2, минимизируя отклонение от исходных значений. Формально:
Минимизировать ||x_new – x_old||, при условии, что близость к центроиду кластера 2 больше, чем к другим

```
import numpy as np
from scipy.optimize import minimize
from sklearn.cluster import KMeans

# Допустим, уже есть обученная модель
kmeans = KMeans(n_clusters=4).fit(X)
centroids = kmeans.cluster_centers_

cluster_2_center = centroids[1]  # Индекс 1 соответствует кластеру 2

def objective(x_new, x_old=client_vector):
    # Функция цели: расстояние до исходного вектора (чем меньше, тем лучше)
    return np.sum((x_new - x_old)**2)

def constraint(x_new):
    # Ограничение: расстояние до кластера 2 меньше, чем до любого другого кластера
    dist_to_2 = np.linalg.norm(x_new - cluster_2_center)
    dists_others = [np.linalg.norm(x_new - c) for i,c in enumerate(centroids) if i != 1]
    return np.min(dists_others) - dist_to_2

# Пример для одного клиента
client_vector = X[0]  # клиент не из кластера 2
cons = ({'type': 'ineq', 'fun': constraint})
result = minimize(objective, x0=client_vector, constraints=cons)
x_new = result.x
```


## 3.

Одна модель с 1000 деревьев предпочтительнее, чем две модели по 500, поскольку объединение результатов двух моделей в среднем даст тот же эффект, что и увеличение числа деревьев в одной модели

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

model1 = RandomForestClassifier(n_estimators=500, random_state=42)
model2 = RandomForestClassifier(n_estimators=500, random_state=43)

model1.fit(X_train, y_train)
model2.fit(X_train, y_train)
y_pred_ensemble = (model1.predict_proba(X_test) + model2.predict_proba(X_test)) / 2
y_pred_ensemble = y_pred_ensemble.argmax(axis=1)

model_single = RandomForestClassifier(n_estimators=1000, random_state=42)
model_single.fit(X_train, y_train)
y_pred_single = model_single.predict(X_test)

print("2", accuracy_score(y_test, y_pred_ensemble))
print("1", accuracy_score(y_test, y_pred_single))


2 0.95
1 0.95


Тут разницы нет, но у одного дерева будет ниже дисперсия + вычисления одного дерева будут лучше с точки зрения памяти и времени

## 4.

1. Обучить KMeans по историческим данным, разделив клиентов на кластеры
2. Определить долю дефолтных клиентов в каждом кластере
3. Для нового клиента определить ближайший кластер (например, по расстоянию до центроида)
4. Вероятностью дефолта считать долю дефолтных клиентов в этом ближайшем кластере

## 5. 

Преобразовать задачу регрессии в классификацию, разделив доход на категории (биннинг). Затем обучить классификационную модель предсказывать категорию дохода. Для получения прогнозируемого значения можно использовать среднее значение в предсказанной категории или вероятностный подход  

In [15]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_regression

X, y = make_regression(n_samples=1000, n_features=10, noise=0.1, random_state=42)
df = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(10)])
df['income'] = y

# Биннинг целевой переменной на 5 категорий
kbin = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')
df['income_bin'] = kbin.fit_transform(df[['income']]).astype(int)

# Разделение на признаки и цель
X = df.drop(['income', 'income_bin'], axis=1)
y = df['income_bin']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"MSE: {mse:.4f}")

df['income_mean'] = df.groupby('income_bin')['income'].transform('mean')
income_means = df[['income_bin', 'income_mean']].drop_duplicates().sort_values('income_bin')

# Создаем словарь бинн -> средний доход
bin_to_income = dict(zip(income_means['income_bin'], income_means['income_mean']))

income_pred = [bin_to_income[bin_] for bin_ in y_pred]
print(f"Пример предсказанных доходов: {income_pred[:5]}")


MSE: 0.5850
Пример предсказанных доходов: [-62.89889796293166, 5.498433644595297, -185.69325075078345, -185.69325075078345, -62.89889796293166]


Можно использовать вероятностный подход, где прогнозируем доход основан на распределении вероятностей по бинам, например, как средневзвешанное

Во всех случаях теряется значение о точности подхода + нужно убрать выбросы иначе после усреднения значение будет смещено к выбросу и нерепрезентативно 